In [369]:
import pickle, json



In [370]:
atis_fn = "/home/yoga/workspace/knodle_hackathon_fork/knodle/hackathon_seq/atis.json"

In [371]:
with open(atis_fn, "rb") as input_file:
    atis_dataset = json.load(input_file, encoding="UTF8")

In [338]:
#with open(atis_fn, "rb") as input_file:
#    atis_dataset = pickle.load(input_file, encoding="UTF8")

In [184]:
atis_dataset.keys()

dict_keys(['label_dict', 'train_sents', 'train_labels', 'vocab', 'test_sents', 'test_labels'])

In [185]:
#s1 = atis_dataset['train_sents'][0]
#word_to_id = atis_dataset['vocab']
#id_to_word = {word_to_id[word]:word for word in word_to_id}
#id_to_word
#[id_to_word[wid] for wid in s1]

In [186]:
#l1 = atis_dataset['train_labels'][0]
#label_to_id = atis_dataset['label_dict']
#id_to_label = {label_to_id[label]:label for label in label_to_id}
#id_to_label

In [187]:
#[(id_to_word[wid], id_to_label[lid]) for (wid, lid) in zip(s1,l1)]

In [188]:
l1 =  None

In [189]:
# knodle/transformation/majority.py:
#    if not use_probabilistic_labels:
#        # convert labels represented as a prob distribution to a single label using majority voting
#        kwargs = {"choose_random_label": True, "other_class_id": other_class_id}
#        noisy_y_train = np.apply_along_axis(probabilies_to_majority_vote, axis=1, arr=noisy_y_train, **kwargs)


In [190]:
data = atis_dataset

In [191]:
import math
train_dev_sents = data["train_sents"] # list of lists
train_dev_labels = data["train_labels"] # list of lists
num_train = math.floor(0.8 * len(train_dev_sents))
train_sents = train_dev_sents[:num_train]
train_labels = train_dev_labels[:num_train]
dev_sents = train_dev_sents[num_train:]
dev_labels = train_dev_labels[num_train:]
test_sents = data["test_sents"]
test_labels = data["test_labels"]
word_to_id = data["vocab"]
label_to_id = data["label_dict"]

In [192]:
UNK_TOKEN = "<UNK>"
PAD_TOKEN = "<PAD>"
VOCAB_SIZE = len(word_to_id)
NUM_LABELS = len(label_to_id)
EMBEDDING_SIZE = 10
HIDDEN_SIZE=10
MAX_LENGTH=20
#KERNEL_SIZE=5

In [193]:
#  We reorganize the data so that "<PAD>" has id 0, "<UNK>" has id 1, and the label 'O' has id 0 (and all words are still represented). Make sure to also change word_to_id and label_to_id.
id_to_word = {word_to_id[word]: word for word in word_to_id}
id_to_label = {label_to_id[label]: label for label in label_to_id}

def switch_idx(sents, i, j):
    D = {i:j, j:i}
    return [[D.get(word, word) for word in sent] for sent in sents]

for i, j in [(word_to_id["<PAD>"], 0), (word_to_id["<UNK>"], 1)]:
    train_sents = switch_idx(train_sents, i, j)
    dev_sents = switch_idx(dev_sents, i, j)
    test_sents = switch_idx(test_sents, i, j)
    
train_labels = switch_idx(train_labels, label_to_id["O"], 0)
dev_labels = switch_idx(dev_labels, label_to_id["O"], 0)
test_labels = switch_idx(test_labels, label_to_id["O"], 0)

word_to_id[id_to_word[0]] = word_to_id["<PAD>"]
word_to_id[id_to_word[1]] = word_to_id["<UNK>"]
label_to_id[id_to_label[0]] = label_to_id["O"]

label_to_id["O"] = 0
word_to_id["<PAD>"] = 0
word_to_id["<UNK>"] = 1

id_to_word = {word_to_id[word]: word for word in word_to_id}
id_to_label = {label_to_id[label]: label for label in label_to_id}

train_labels

[[0, 0, 0, 0, 0, 48, 0, 35, 99, 0, 0, 0, 78, 0, 14, 0, 0, 12],
 [0, 0, 0, 0, 0, 48, 0, 78, 0, 26, 33],
 [0, 0, 0, 45, 107, 0, 48, 109, 0, 0, 35, 99, 0, 0, 48],
 [21, 0, 0, 48, 0, 78],
 [66, 117, 0, 0, 48, 0, 78, 21, 38, 102],
 [0, 0, 0, 0, 29, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 48, 0, 78, 123, 0, 26],
 [0, 0, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 0, 0, 17],
 [0, 0, 0, 0, 0, 48, 0, 78, 123],
 [0, 0, 0, 0, 0, 48, 109, 0, 78, 0, 0, 0, 71],
 [0, 0, 0, 0, 0, 17],
 [0, 0, 0, 21, 0, 0, 48, 0, 76],
 [0, 0, 0, 78, 36, 35, 99],
 [0, 0, 0, 18, 92, 0, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 17],
 [0, 0, 0, 48, 0, 78, 0, 36, 35, 99, 0, 36, 35, 99],
 [0, 0, 0, 0, 0, 0, 26, 0, 48, 0, 78, 48, 0, 78, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 48, 0, 78, 0, 26, 0, 25, 0],
 [0, 0, 0, 0, 0, 0, 48, 0, 78, 0, 2, 83],
 [0, 0, 0, 0, 0, 48, 109, 0, 78],
 [0, 0, 0, 18, 92, 0, 0, 2, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 0, 2, 83],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 0, 78, 0, 78],
 [0, 0

In [194]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [195]:
import torch
def do_padding(sequences, length = MAX_LENGTH):
    return pad_sequences(sequences, maxlen = length)

train_sents_padded = do_padding(train_sents)
dev_sents_padded = do_padding(dev_sents)
test_sents_padded = do_padding(test_sents)

train_labels_padded = torch.LongTensor(do_padding(train_labels))
dev_labels_padded = do_padding(dev_labels)
test_labels_padded = do_padding(test_labels)

# TODO: if we want 1-hot vectors:
#train_labels_padded = to_categorical(do_padding(train_labels), NUM_LABELS)
#dev_labels_padded = to_categorical(do_padding(dev_labels), NUM_LABELS)
#test_labels_padded = to_categorical(do_padding(test_labels), NUM_LABELS)

In [196]:
train_sents_padded

array([[  0,   0, 232, ..., 234, 481, 321],
       [  0,   0,   0, ..., 358, 496, 321],
       [  0,   0,   0, ..., 193, 270, 543],
       ...,
       [  0,   0,   0, ..., 237, 200, 137],
       [  0,   0,   0, ...,  85, 502, 137],
       [  0,   0,   0, ..., 100, 502, 236]], dtype=int32)

In [197]:
#https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling/blob/master/dynamic_rnn.py

In [198]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [199]:
# Create an embedding layer, with 0 vectors for the pads
embeds = nn.Embedding(len(word_to_id), 10, padding_idx=0)

# Create an LSTM layer
lstm = nn.LSTM(10, 10, bidirectional=False, batch_first=True)

In [200]:
seqs = torch.LongTensor(train_sents_padded)
embeddings = embeds(seqs)
out_static, _ = lstm(embeddings)
assert out_static.size(1) == embeddings.size(1)

In [201]:
#seq_lens=...
#packed_seqs = pack_padded_sequence(embeddings, seq_lens.tolist(), batch_first=True)

In [225]:
# https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling/blob/master/train.py
class LstmModel(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, padding_idx=0):
        super(LstmModel, self).__init__()
        self.tagset_size = tagset_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        self.embeds = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=False, batch_first=True)
        self.emission = nn.Linear(hidden_dim, tagset_size)
    
    def forward(self, token_idxs):
        embeds = self.embeds(token_idxs)
        hidden, _ = self.lstm(embeds)
        pred = self.emission(hidden)
        return pred

In [227]:
s_batch = torch.LongTensor(train_sents_padded[0:2])
s_batch

tensor([[  0,   0, 232, 542, 502, 196, 208,  77,  62,  10,  35,  40,  58, 234,
         137,  62,  11, 234, 481, 321],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0, 554, 194,  50,  66, 208,
         379, 502,  69, 358, 496, 321]])

In [228]:
out = model(s_batch)

In [229]:
out[0].shape

torch.Size([20, 127])

In [230]:
from torch.utils.data import Dataset

class SeqDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels
        self.data_size = len(self.tokens)

    def __getitem__(self, i):
        return self.tokens[i], self.labels[i]

    def __len__(self):
        return self.data_size

In [231]:
def adjust_learning_rate(optimizer, new_lr):
    """
    Shrinks learning rate by a specified factor.
    :param optimizer: optimizer whose learning rates must be decayed
    :param new_lr: new learning rate
    """

    print("\nDECAYING learning rate.")
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr
    print("The new learning rate is %f\n" % (optimizer.param_groups[0]['lr'],))

In [232]:
# Training parameters
start_epoch = 0  # start at this epoch
batch_size = 10  # batch size
lr = 0.015  # learning rate
lr_decay = 0.05  # decay learning rate by this amount
momentum = 0.9  # momentum
workers = 1  # number of workers for loading data in the DataLoader
epochs = 20  # number of epochs to run without early-stopping
grad_clip = 5.  # clip gradients at this value
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [233]:
train_sents_padded.shape

(3982, 20)

In [234]:
def clip_gradient(optimizer, grad_clip):
    """
    Clip gradients computed during backpropagation to prevent gradient explosion.
    :param optimizer: optimized with the gradients to be clipped
    :param grad_clip: gradient clip value
    """
    for group in optimizer.param_groups:
        for param in group['params']:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

In [235]:
def accuracy_padded(predicted, gold, mask):
    return sum(sum((predicted == gold)* mask) ) / sum(sum(mask))

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [55]:



train_labels

[[0, 0, 0, 0, 0, 48, 0, 35, 99, 0, 0, 0, 78, 0, 14, 0, 0, 12],
 [0, 0, 0, 0, 0, 48, 0, 78, 0, 26, 33],
 [0, 0, 0, 45, 107, 0, 48, 109, 0, 0, 35, 99, 0, 0, 48],
 [21, 0, 0, 48, 0, 78],
 [66, 117, 0, 0, 48, 0, 78, 21, 38, 102],
 [0, 0, 0, 0, 29, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 48, 0, 78, 123, 0, 26],
 [0, 0, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 0, 0, 17],
 [0, 0, 0, 0, 0, 48, 0, 78, 123],
 [0, 0, 0, 0, 0, 48, 109, 0, 78, 0, 0, 0, 71],
 [0, 0, 0, 0, 0, 17],
 [0, 0, 0, 21, 0, 0, 48, 0, 76],
 [0, 0, 0, 78, 36, 35, 99],
 [0, 0, 0, 18, 92, 0, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 17],
 [0, 0, 0, 48, 0, 78, 0, 36, 35, 99, 0, 36, 35, 99],
 [0, 0, 0, 0, 0, 0, 26, 0, 48, 0, 78, 48, 0, 78, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 48, 0, 78, 0, 26, 0, 25, 0],
 [0, 0, 0, 0, 0, 0, 48, 0, 78, 0, 2, 83],
 [0, 0, 0, 0, 0, 48, 109, 0, 78],
 [0, 0, 0, 18, 92, 0, 0, 2, 0, 48, 0, 78],
 [0, 0, 0, 0, 0, 0, 0, 2, 83],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 0, 78, 0, 78],
 [0, 0

In [56]:
lf_to_newlabel = dict()

for oldlabel in label_to_id.keys():
    #print(label)
    #print(label.split(".")[0].split("-")[-1])
    newlabel = oldlabel.split(".")[0].split("-")[-1]
    lf_to_newlabel[oldlabel] = newlabel

del lf_to_newlabel["O"]

In [57]:
lf_to_newlabel

{'I-depart_date.today_relative': 'depart_date',
 'B-cost_relative': 'cost_relative',
 'B-flight': 'flight',
 'I-state_name': 'state_name',
 'I-flight_stop': 'flight_stop',
 'B-mod': 'mod',
 'B-fromloc.airport_name': 'fromloc',
 'B-restriction_code': 'restriction_code',
 'B-return_date.month_name': 'return_date',
 'I-flight_time': 'flight_time',
 'B-economy': 'economy',
 'B-depart_date.year': 'depart_date',
 'B-state_name': 'state_name',
 'B-depart_date.date_relative': 'depart_date',
 'B-airline_name': 'airline_name',
 'I-toloc.city_name': 'toloc',
 'B-arrive_time.start_time': 'arrive_time',
 'B-airport_name': 'airport_name',
 'B-flight_mod': 'flight_mod',
 'I-today_relative': 'today_relative',
 'B-stoploc.airport_code': 'stoploc',
 'I-flight_number': 'flight_number',
 'B-or': 'or',
 'B-depart_time.time': 'depart_time',
 'I-flight_mod': 'flight_mod',
 'B-arrive_date.today_relative': 'arrive_date',
 'I-transport_type': 'transport_type',
 'B-depart_time.start_time': 'depart_time',
 'B-ret

In [58]:
print(len(set(lf_to_newlabel.keys())))
print(len(set(lf_to_newlabel.values())))

126
47


In [59]:
# TODO:
# 1. use original labels, without "O" to represent Z-matrix
# 2. make T matrix
# 3. do majority vote
# 4. make new label vector
# 5. train
# 6. check result of training

In [251]:
# 1. use original labels, without "O" to represent Z-matrix
train_labels_padded = to_categorical(do_padding(train_labels), NUM_LABELS)[:,:,1:]
dev_labels_padded = to_categorical(do_padding(dev_labels), NUM_LABELS)[:,:,1:]
test_labels_padded = to_categorical(do_padding(test_labels), NUM_LABELS)[:,:,1:]

In [252]:
sorted(list(id_to_label.items()))

[(0, 'O'),
 (1, 'B-airline_code'),
 (2, 'B-airline_name'),
 (3, 'B-airport_code'),
 (4, 'B-airport_name'),
 (5, 'B-arrive_date.date_relative'),
 (6, 'B-arrive_date.day_name'),
 (7, 'B-arrive_date.day_number'),
 (8, 'B-arrive_date.month_name'),
 (9, 'B-arrive_date.today_relative'),
 (10, 'B-arrive_time.end_time'),
 (11, 'B-arrive_time.period_mod'),
 (12, 'B-arrive_time.period_of_day'),
 (13, 'B-arrive_time.start_time'),
 (14, 'B-arrive_time.time'),
 (15, 'B-arrive_time.time_relative'),
 (16, 'B-booking_class'),
 (17, 'B-city_name'),
 (18, 'B-class_type'),
 (19, 'B-compartment'),
 (20, 'B-connect'),
 (21, 'B-cost_relative'),
 (22, 'B-day_name'),
 (23, 'B-day_number'),
 (24, 'B-days_code'),
 (25, 'B-depart_date.date_relative'),
 (26, 'B-depart_date.day_name'),
 (27, 'B-depart_date.day_number'),
 (28, 'B-depart_date.month_name'),
 (29, 'B-depart_date.today_relative'),
 (30, 'B-depart_date.year'),
 (31, 'B-depart_time.end_time'),
 (32, 'B-depart_time.period_mod'),
 (33, 'B-depart_time.perio

In [253]:
newlabel_to_id = dict()
OTHER_ID = 0
OTHER_LABEL = "O"
newlabel_to_id[OTHER_LABEL] = OTHER_ID

for i, nl in enumerate(set(lf_to_newlabel.values())):
    newlabel_to_id[nl] = i + 1

In [254]:
lfid_to_nlid = dict()

In [255]:
for lf_id in id_to_label:
    if lf_id == 0:
        continue
    lf_name = id_to_label[lf_id]
    newlabel_name = lf_to_newlabel[lf_name]
    nl_id = newlabel_to_id[newlabel_name]
    # "O" is not a lf in the 1-hot encoding of the original labels
    lfid_to_nlid[lf_id - 1] = nl_id

In [256]:
lfid_to_nlid

{94: 24,
 20: 9,
 39: 28,
 117: 23,
 105: 13,
 53: 42,
 46: 46,
 57: 18,
 61: 14,
 106: 37,
 36: 1,
 29: 24,
 67: 23,
 24: 24,
 1: 8,
 122: 41,
 12: 4,
 3: 25,
 41: 19,
 120: 2,
 68: 5,
 104: 11,
 55: 26,
 34: 29,
 103: 19,
 8: 6,
 124: 33,
 33: 29,
 59: 14,
 56: 45,
 118: 5,
 52: 30,
 101: 44,
 30: 29,
 70: 5,
 100: 1,
 80: 41,
 43: 13,
 75: 41,
 13: 4,
 60: 14,
 26: 24,
 97: 29,
 107: 46,
 19: 20,
 9: 4,
 66: 31,
 111: 30,
 65: 15,
 2: 3,
 64: 16,
 69: 5,
 109: 46,
 51: 43,
 116: 15,
 74: 2,
 45: 46,
 113: 14,
 7: 6,
 32: 29,
 112: 18,
 72: 34,
 110: 43,
 85: 4,
 38: 7,
 5: 6,
 6: 6,
 96: 29,
 108: 46,
 15: 27,
 78: 41,
 99: 29,
 28: 24,
 4: 6,
 11: 4,
 91: 17,
 77: 41,
 47: 46,
 123: 41,
 87: 4,
 83: 25,
 17: 17,
 81: 33,
 71: 5,
 22: 21,
 73: 22,
 16: 32,
 86: 4,
 92: 9,
 58: 14,
 90: 32,
 84: 6,
 89: 4,
 48: 46,
 54: 38,
 50: 12,
 63: 16,
 37: 44,
 125: 39,
 114: 14,
 115: 14,
 95: 29,
 23: 47,
 49: 46,
 119: 34,
 102: 7,
 62: 14,
 88: 4,
 18: 10,
 14: 4,
 44: 37,
 76: 41,
 27: 24

In [262]:
import numpy as np
#t_matrix = torch.tensor(np.zeros((len(lfid_to_nlid), len(newlabel_to_id))))
t_matrix = np.zeros((len(lfid_to_nlid), len(newlabel_to_id)))

In [263]:
for lfid in lfid_to_nlid:
    t_matrix[lfid, lfid_to_nlid[lfid]] = 1
t_matrix[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [264]:
train_labels[0]

[0, 0, 0, 0, 0, 48, 0, 35, 99, 0, 0, 0, 78, 0, 14, 0, 0, 12]

In [292]:
print(train_labels_padded.shape)
print(t_matrix.shape)
# train_labels_padded <-> "Z matrix" <-> lf matches <-> original fine-grained 'labels'
print(train_labels_padded.dot(t_matrix).shape)
train_newlabels_padded = train_labels_padded.dot(t_matrix)

(3982, 20, 126)
(126, 48)
(3982, 20, 48)


In [298]:
mv_vector = np.ones(len(newlabel_to_id))
mv_vector[OTHER_ID] = 0

lf_match = train_newlabels_padded.dot(mv_vector)

lf_match.shape

train_newlabels_padded[lf_match==0, OTHER_ID] = 1


In [295]:
train_newlabels_padded[0, 0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [297]:
has_label = lf_match#train_newlabels_padded.max(axis=2)
has_label

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [307]:
#label_ids = (train_newlabels_padded.argmax(axis=2) + 1) * has_label - 1

label_ids = train_newlabels_padded.argmax(axis=2) 


sum(label_ids != label_ids * has_label)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [319]:
id_to_newlabel = {i:l for l,i in newlabel_to_id.items()}
id_to_newlabel

{0: 'O',
 1: 'economy',
 2: 'today_relative',
 3: 'airport_code',
 4: 'arrive_time',
 5: 'stoploc',
 6: 'arrive_date',
 7: 'fare_basis_code',
 8: 'airline_name',
 9: 'cost_relative',
 10: 'compartment',
 11: 'flight_number',
 12: 'meal',
 13: 'flight_stop',
 14: 'return_date',
 15: 'round_trip',
 16: 'return_time',
 17: 'class_type',
 18: 'restriction_code',
 19: 'flight_mod',
 20: 'connect',
 21: 'day_number',
 22: 'time_relative',
 23: 'state_name',
 24: 'depart_date',
 25: 'airport_name',
 26: 'or',
 27: 'booking_class',
 28: 'flight',
 29: 'depart_time',
 30: 'meal_description',
 31: 'state_code',
 32: 'city_name',
 33: 'transport_type',
 34: 'time',
 35: 'day_name',
 36: 'airline_code',
 37: 'flight_time',
 38: 'month_name',
 39: 'aircraft_code',
 40: 'flight_days',
 41: 'toloc',
 42: 'mod',
 43: 'meal_code',
 44: 'fare_amount',
 45: 'period_of_day',
 46: 'fromloc',
 47: 'days_code'}

In [309]:
nl1 = label_ids[0]
#[(id_to_word[wid], id_to_label[lid]) for (wid, lid) in zip(s1,nl1)]
nl1

array([ 0,  0,  0,  0,  0,  0,  0, 46,  0, 29, 29,  0,  0,  0, 41,  0,  4,
        0,  0,  4])

In [310]:
s1 = train_sents_padded[0]
id_to_word
nl1

array([ 0,  0,  0,  0,  0,  0,  0, 46,  0, 29, 29,  0,  0,  0, 41,  0,  4,
        0,  0,  4])

In [311]:
[(id_to_word[wid], id_to_newlabel[lid]) for (wid, lid) in zip(s1, nl1)]
#[id_to_word[wid] for wid in s1]

[('<PAD>', 'O'),
 ('<PAD>', 'O'),
 ('i', 'O'),
 ('want', 'O'),
 ('to', 'O'),
 ('fly', 'O'),
 ('from', 'O'),
 ('boston', 'fromloc'),
 ('at', 'O'),
 ('DIGITDIGITDIGIT', 'depart_time'),
 ('am', 'depart_time'),
 ('and', 'O'),
 ('arrive', 'O'),
 ('in', 'O'),
 ('denver', 'toloc'),
 ('at', 'O'),
 ('DIGITDIGITDIGITDIGIT', 'arrive_time'),
 ('in', 'O'),
 ('the', 'O'),
 ('morning', 'arrive_time')]

In [312]:
def accuracy(predicted, gold):
    return sum(sum(predicted == gold)) / gold.size

def accuracy_padded(predicted, gold, mask):
    return sum(sum((predicted == gold) * mask)) / sum(sum(mask))

In [313]:
pred_test = nl1.copy()
pred_test[-2] += 1
pred_test[-1] += 1

accuracy(np.vstack([pred_test, pred_test]),np.vstack([nl1, nl1]))

mask = (s1!=0)

accuracy_padded(np.vstack([pred_test, pred_test]),np.vstack([nl1, nl1]), np.vstack([mask, mask]))


0.8888888888888888

In [314]:
nl1

array([ 0,  0,  0,  0,  0,  0,  0, 46,  0, 29, 29,  0,  0,  0, 41,  0,  4,
        0,  0,  4])

In [346]:
def majority_vote(lf_matches, t_matrix, OTHER_ID = 0):
    ws_labels = lf_matches.dot(t_matrix)
    num_labels = t_matrix.shape[1]
    
    # Count all lf matches
    mv_vector = np.ones(num_labels)
    mv_vector[OTHER_ID] = 0
    lf_match = ws_labels.dot(mv_vector)
    
    # If no lf match, assign OTHER class
    ws_labels[lf_match==0, OTHER_ID] = 1
    
    # Divide by total count (last axis)
    ws_labels / ws_labels.sum(axis=2)[..., None]
    return ws_labels

In [347]:
majority_vote(train_labels_padded, t_matrix)

array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [366]:
train_lf_matches = train_labels_padded
print(train_lf_matches.shape)
print(t_matrix.shape)
dev_labels = majority_vote(dev_labels_padded, t_matrix).argmax(axis=2)
print(dev_labels.shape)
print(train_sents_padded.shape)
print(dev_sents_padded.shape)


(3982, 20, 126)
(126, 48)
(996, 20)
(3982, 20)
(996, 20)


In [358]:
#len(label_to_id)

num_labels = t_matrix.shape[1]

model = LstmModel(vocab_size = len(word_to_id), tagset_size = num_labels, embedding_dim=30, hidden_dim=50)

In [368]:
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)

criterion = nn.CrossEntropyLoss().to(device)


train_mapped_labels = majority_vote(train_lf_matches, t_matrix).argmax(axis=2)

train_loader = torch.utils.data.DataLoader(SeqDataset(train_sents_padded, train_mapped_labels), batch_size=batch_size, shuffle=True,
                                               num_workers=workers, pin_memory=False)

#train_loader = torch.utils.data.DataLoader(SeqDataset(train_sents_padded, train_labels_padded), batch_size=batch_size, shuffle=True,
#                                               num_workers=workers, pin_memory=False)


#val_loader = torch.utils.data.DataLoader(SeqDataset(dev_sents_padded, dev_labels_padded), batch_size=batch_size, shuffle=True,
#                                             num_workers=workers, pin_memory=False)

val_loader = torch.utils.data.DataLoader(SeqDataset(dev_sents_padded, dev_labels), batch_size=batch_size, shuffle=True,
                                             num_workers=workers, pin_memory=False)

print_val = True

# Epochs
for epoch in range(start_epoch, epochs):
    model.train()
    
    if print_val:
        avg_acc = 0
        for i, (tokens, labels_) in enumerate(val_loader):
            preds_ = model(tokens)
            preds = preds_.argmax(axis=2)
            #labels = labels_.argmax(axis=2)
            mask = (tokens != 0)
            acc = accuracy_padded(predicted=preds , gold=labels_ , mask=mask)
            avg_acc = (avg_acc*i + acc) / (i+1)
        print(avg_acc)

    for i, (tokens, labels) in enumerate(train_loader):
    #for (tokens, labels) in train_loader:
        
#        print(tokens.shape)
#        print(s_batch.shape)
#        print(train_sents_padded.shape)
        
        #logits = model(s_batch)
        #print(logits.shape)

        # batchsize x nb_classes x seq_len
        logits = model(tokens).permute(0,2,1)
        #print(logits.shape)
        #print(labels.shape)
        
        #labels = torch.LongTensor(labels)
        
        
        loss = criterion(logits, labels)
                # Back prop.
        optimizer.zero_grad()
        loss.backward()

        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        optimizer.step()

    # One epoch's validation
    #val_f1 = validate(val_loader=val_loader,
    #                  model=model,
    #                  crf_criterion=crf_criterion,
    #                  vb_decoder=vb_decoder)    
    adjust_learning_rate(optimizer, lr / (1 + (epoch + 1) * lr_decay))


tensor(0.9508)

DECAYING learning rate.
The new learning rate is 0.014286

tensor(0.9522)

DECAYING learning rate.
The new learning rate is 0.013636

tensor(0.9523)

DECAYING learning rate.
The new learning rate is 0.013043

tensor(0.9546)

DECAYING learning rate.
The new learning rate is 0.012500

tensor(0.9568)

DECAYING learning rate.
The new learning rate is 0.012000

tensor(0.9588)

DECAYING learning rate.
The new learning rate is 0.011538

tensor(0.9600)

DECAYING learning rate.
The new learning rate is 0.011111

tensor(0.9615)

DECAYING learning rate.
The new learning rate is 0.010714

tensor(0.9627)

DECAYING learning rate.
The new learning rate is 0.010345

tensor(0.9632)

DECAYING learning rate.
The new learning rate is 0.010000

tensor(0.9640)

DECAYING learning rate.
The new learning rate is 0.009677

tensor(0.9644)

DECAYING learning rate.
The new learning rate is 0.009375

tensor(0.9661)

DECAYING learning rate.
The new learning rate is 0.009091

tensor(0.9654)

DECAYING 